<h2>Part 1</h2>

In [113]:
import requests

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge


In [114]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_random_wikipedia_page = requests.get(wikipedia_link)
page = raw_random_wikipedia_page.text

start = page.find('<table class="wikitable sortable">')
finish = page.find('</table>')
table = page[start+34:finish]

In [115]:
colList = ["PostalCode", "Borough", "Neighbourhood"]
rowList = []

bs=BeautifulSoup(table, "lxml")
table_body=bs.find('tbody')
rows = table_body.find_all('tr')
count = 0
for row in rows:
    #count
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    rowList.append(cols)

df_table = pd.DataFrame(rowList, columns = colList)
df_table.head()

,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [116]:
df_table.dropna(inplace=True) 
df_table.drop(df_table[df_table.Borough == "Not assigned"].index, axis=0, inplace=True)
df_table.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [117]:
df_table = df_table.reset_index(drop=True)

#getting list of indices where Neighbourhoods are "Not assigned"
indexesWithNoNeighbouhood = df_table[df_table.Neighbourhood == "Not assigned"].index

#replacing all 'Not Assigned' Neghbourhoods with their Borough Name
for indexesWithNoNeighbouhood in indexesWithNoNeighbouhood:
    df_table.replace(df_table.iat[indexesWithNoNeighbouhood,2] ,df_table.iat[indexesWithNoNeighbouhood,1], inplace = True)

In [118]:
df_table=df_table.groupby(['PostalCode']).agg({'Borough': 'first','Neighbourhood':', '.join})
df_table.reset_index(inplace=True)
df_table = df_table[colList]
df_table.shape

(103, 3)

In [119]:
df_table.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [120]:
df_table.shape

(103, 3)